# Updating SharePoint document colums using AI

This notebook demonstrates how to use AI to automatically extract metadata from SharePoint documents, using structured outputs. While we'll use legal documents as examples, this approach works for any scenario where manual metadata entry is a burden.

## The Metadata Problem

Document management systems like SharePoint rely heavily on metadata (columns/fields) for:
- **Search**: Find documents by properties rather than just text
- **Organization**: Group related documents automatically
- **Workflows**: Trigger actions based on document properties
- **Compliance**: Track and enforce document policies

However, users often avoid filling in metadata fields because:
1. It's time-consuming
2. The fields' purpose isn't always clear
3. They'd rather save to their desktop than fill out forms

This leads to:
- Poor searchability
- Duplicate content
- Missed automation opportunities

## The Solution: Structured AI Outputs

Instead of asking users to fill in metadata, we can use AI to automatically extract it. The key is using structured outputs rather than chat-style interactions:

In [ ]:
# Define what metadata we want to extract
$schema = @{
    name = "legal_document_metadata"
    strict = $true
    schema = @{
        type = "object"
        properties = @{
            document_type = @{
                type = "string"
                enum = @(
                    "Case Brief", "Legal Opinion", "Memo",
                    "Article", "Regulation", "Statute",
                    "Petition", "News"
                )
                description = "What type of legal document is this?"
            }
            subject_area = @{
                type = "string"
                enum = @(
                    "Constitutional Law", "Tax Law", "Environmental Law",
                    "Employment Law", "Intellectual Property", "Family Law"
                )
                description = "What area of law does this document concern?"
            }
            parties = @{
                type = "array"
                items = @{ type = "string" }
                description = "Who are the main parties mentioned in this document?"
            }
            priority = @{
                type = "string"
                enum = @("High", "Medium", "Low")
                description = "What is the priority level of this document?"
            }
            keywords = @{
                type = "array"
                items = @{ type = "string" }
                description = "What are the key legal concepts or terms in this document?"
            }
            requires_action = @{
                type = "boolean"
                description = "Does this document require immediate action or response?"
            }
        }
        required = @(
            "document_type", "subject_area", "parties",
            "keywords", "requires_action"
        )
    }
}

This schema maps directly to SharePoint columns:
- Choice columns for document_type, subject_area, priority
- Managed metadata for keywords
- Multi-line text for parties (could be Person column type)
- Yes/No column for requires_action

## Real-World Examples

### Example 1: Update Existing SharePoint Library

This example shows how to enhance an existing document library with AI-extracted metadata. Perfect for libraries with lots of documents but minimal metadata:

In [ ]:
# First, connect to SharePoint (requires PnP PowerShell)
Connect-PnPOnline -Url "https://contoso.sharepoint.com/sites/legal" -Interactive

# Get all documents from a specific library
$docs = Get-PnPListItem -List "Documents" -PageSize 500

foreach ($doc in $docs) {
    try {
        # Download file to temp location
        $tempPath = Join-Path $env:TEMP $doc.FieldValues.FileLeafRef
        Get-PnPFile -Url $doc.FieldValues.FileRef -Path $env:TEMP -Filename $doc.FieldValues.FileLeafRef -AsFile

        # Extract metadata using our schema
        $metadata = Get-Item $tempPath |
                    Invoke-DocProcessing -Schema $schema

        # Update SharePoint item with new metadata
        $updateValues = @{
            DocumentType = $metadata.document_type
            SubjectArea = $metadata.subject_area
            Parties = $metadata.parties -join "; "
            Priority = $metadata.priority
            Keywords = $metadata.keywords # Assumes managed metadata field
            RequiresAction = $metadata.requires_action
        }

        Set-PnPListItem -List "Documents" -Identity $doc.Id -Values $updateValues
        Write-Host "Updated metadata for: $($doc.FieldValues.FileLeafRef)"

        # Cleanup
        Remove-Item $tempPath -Force
    }
    catch {
        Write-Warning "Failed to process $($doc.FieldValues.FileLeafRef): $_"
    }
}

# Disconnect from SharePoint
Disconnect-PnPOnline

This script:
1. Connects to your SharePoint site
2. Gets all documents from a library
3. Downloads each file temporarily
4. Extracts metadata using AI
5. Updates the SharePoint item
6. Cleans up temporary files

Great for:
- Enhancing existing document libraries
- Standardizing metadata across documents
- Making old documents more searchable

## Processing a Single Document

Let's look at how the metadata extraction works on a single document:

In [ ]:
# Process a single document
Get-ChildItem "./docs/Case Brief Jaques Dubois.docx" |
    Invoke-DocProcessing -Schema $schema

Notice how the AI:
1. Reads the document content
2. Extracts relevant metadata
3. Returns it in exactly the format we specified

## Batch Processing

The real power comes from processing multiple documents. Here we'll process EPA regulations and tax memos to show how it handles different document types:

In [ ]:
# Process multiple documents
Get-ChildItem './docs/News EPA Regulations.docx', './docs/Memo Tax Foreign Investments.docx' |
    Invoke-DocProcessing -Schema $schema

## More Integration Examples

### SharePoint Upload with Metadata
```powershell
# Example SharePoint upload with metadata (requires PnP PowerShell)
$results = Get-ChildItem ./docs |
           Invoke-DocProcessing -Schema $schema

foreach ($doc in $results) {
    Add-PnPFile -Path $doc.Path -NewFileName $doc.Name -Values @{
        DocumentType = $doc.Metadata.document_type
        SubjectArea = $doc.Metadata.subject_area
        Parties = $doc.Metadata.parties -join "; "
        Priority = $doc.Metadata.priority
        Keywords = $doc.Metadata.keywords
        RequiresAction = $doc.Metadata.requires_action
    }
}
```

### DevOps Pipeline Integration
```yaml
# Example Azure Pipeline task
- task: PowerShell@2
  inputs:
    targetType: 'inline'
    script: |
      $schema = Get-Content ./metadata-schema.json | ConvertFrom-Json
      Get-ChildItem ./docs -Recurse |
        Invoke-DocProcessing -Schema $schema |
        Export-Csv -Path ./metadata.csv
```

## Why This Matters

1. **Zero User Input**: Metadata is extracted automatically
2. **Consistent Results**: Schema ensures standardized output
3. **Flexible Integration**: Works with any document management system
4. **Scalable Automation**: Process thousands of documents easily

Think about how much time this saves:
- Users just upload documents normally
- AI extracts metadata automatically
- Documents become instantly searchable
- Workflows can trigger based on metadata

And because we're using structured outputs instead of chat:
- Results are consistent and validated
- Output maps directly to SharePoint columns
- Easy to integrate with existing systems
- Scales to handle large document libraries

## Try it Now

Let's try a comprehensive example with a new legal document schema that captures even more metadata.

### Before You Begin

Before running the examples in this notebook, make sure you have:

1. Set the required environment variables:
   - `API_KEY`: Your OpenAI API key
   - `API_BASE`: (Optional) API base URL if using Azure OpenAI or other providers
   - `Model`: The model to use (e.g., "gpt-4o", "gpt-4o-mini")
   - `AZURE_FUNCTION_KEY`: Function key for authentication

2. The PowerShell modules required for document processing are installed

3. If using Azure OpenAI or other AI providers, ensure your `API_BASE` is properly configured (e.g., https://xyz.openai.azure.com for Azure OpenAI)

4. Start the Azure Function shell in VS Code Terminal.

The environment variables can be set in your PowerShell session or added to your local.settings.json file.

Once you've got those setup...

In [ ]:
# Define what metadata we want to extract
$schema = @{
    name = "legal_document_metadata"
    strict = $true
    schema = @{
        type = "object"
        properties = @{
            area_of_law = @{
                type = "string"
                enum = @(
                    "Criminal Law", "Civil Law", "Contract Law",
                    "Property Law", "Family Law", "Labor and Employment Law",
                    "Intellectual Property Law", "Tax Law", "Constitutional Law",
                    "Administrative Law"
                )
                description = "What area of law does the document primarily belong to?"
            }
            document_type = @{
                type = "string"
                enum = @(
                    "Case Brief", "Legal Opinion", "Contract",
                    "Statute", "Regulation", "Court Filing",
                    "Legal Memorandum", "Legal Article", "Legal News"
                )
                description = "What type of legal document is this?"
            }
            parties_involved = @{
                type = "array"
                items = @{ type = "string" }
                description = "What are the names of the parties involved in this legal matter?"
            }
            court = @{
                type = "string"
                description = "Which court or jurisdiction does this document relate to?"
            }
            case_number = @{
                type = "string"
                description = "What case number is associated with this document, if applicable."
            }
            keywords = @{
                type = "array"
                items = @{ type = "string" }
                description = "What are some good keywords or tags for this document?"
            }
        }
        required = @(
            "area_of_law", "document_type", "parties_involved",
            "court", "case_number", "keywords"
        )
        additionalProperties = $false
    }
}


# Process all documents in the docs directory
Get-ChildItem /workspaces/soaps/azure-function/docs |
    Invoke-DocProcessing -Schema $schema

# Power Platform

The Power Platform integrates our metadata extraction solution directly into SharePoint workflows.

![powerplatform.png](powerplatform.png)

As seen in the image, we can create a Power Automate flow with these steps:

1. **Automatic trigger** when a new document is uploaded to SharePoint ("When a file is created (properties only)")
2. **File content retrieval** from the document library
3. **POST request to Azure Function** to process the document
4. **Structured output parsing** from the Azure Function response
5. **SharePoint property updates** with the extracted metadata

This automation eliminates manual metadata entry:
- Users upload documents to SharePoint through standard methods
- The flow processes each document in the background
- Metadata columns are populated automatically
- Documents become properly indexed for search and organization

The flow can be configured to:
- Work with specific document libraries
- Use different schemas for various document types
- Alert users about important documents
- Initiate additional processes based on document metadata

This approach connects SharePoint for document storage, Azure Functions for processing, and Power Automate for workflow orchestration - creating a complete solution that reduces manual work while improving document organization and discoverability.